In [17]:
#import needed libraries、
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from autoencoder_pipeline import train_autoencoder
from detection import train_vae_anomaly_detection
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import lightgbm as lgb
import xgboost as xgb
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler



# 选择设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 数据准备

In [18]:
df_application_record = pd.read_csv("application_record.csv")
df_credit_record = pd.read_csv("credit_record.csv")

print(df_application_record.info())
print(df_application_record.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

In [19]:
#For each set of duplicate ID's drop both of them
df_application_record = df_application_record.drop_duplicates(subset = 'ID', keep = False)
df_application_record.info()

<class 'pandas.core.frame.DataFrame'>
Index: 438463 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438463 non-null  int64  
 1   CODE_GENDER          438463 non-null  object 
 2   FLAG_OWN_CAR         438463 non-null  object 
 3   FLAG_OWN_REALTY      438463 non-null  object 
 4   CNT_CHILDREN         438463 non-null  int64  
 5   AMT_INCOME_TOTAL     438463 non-null  float64
 6   NAME_INCOME_TYPE     438463 non-null  object 
 7   NAME_EDUCATION_TYPE  438463 non-null  object 
 8   NAME_FAMILY_STATUS   438463 non-null  object 
 9   NAME_HOUSING_TYPE    438463 non-null  object 
 10  DAYS_BIRTH           438463 non-null  int64  
 11  DAYS_EMPLOYED        438463 non-null  int64  
 12  FLAG_MOBIL           438463 non-null  int64  
 13  FLAG_WORK_PHONE      438463 non-null  int64  
 14  FLAG_PHONE           438463 non-null  int64  
 15  FLAG_EMAIL           4

In [20]:
#show how many unique IDs we will be able to work with in the dataframes
print("# of unique IDs that are consistent between both datasets", df_application_record[df_application_record['ID'].isin(df_credit_record['ID'])]['ID'].nunique())

#adjust the dataframes so that we only work with the consistent IDs
df_application_record = df_application_record[df_application_record['ID'].isin(df_credit_record['ID'])]
df_credit_record = df_credit_record[df_credit_record['ID'].isin(df_application_record['ID'])]
print("New # of IDs in application_record", df_application_record['ID'].nunique())
print("New # of IDs in credit_record", df_credit_record['ID'].nunique())

# of unique IDs that are consistent between both datasets 36457
New # of IDs in application_record 36457
New # of IDs in credit_record 36457


# 数据清洗

In [21]:
df_credit_record['APPROVED'] = df_credit_record['STATUS'].map({'1':0,'2':0,'3':0,'4':0,'5':0,'X':-1,'C':1,'0':1})
df_credit_record = df_credit_record[df_credit_record['APPROVED']!=-1]
df_credit_record['STATUS'].value_counts()

STATUS
C    329536
0    290654
1      8747
5      1527
2       801
3       286
4       214
Name: count, dtype: int64

In [22]:
df_application_record = df_application_record.merge(df_credit_record, on='ID')
df_application_record.head(10)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS,APPROVED
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,0,C,1
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-1,C,1
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-2,C,1
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-3,C,1
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-4,C,1
5,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-5,C,1
6,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-6,C,1
7,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-7,C,1
8,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-8,C,1
9,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,-9,C,1


In [23]:
df_application_record = df_application_record[df_application_record['MONTHS_BALANCE']==-4]
df_application_record.loc[df_application_record["NAME_INCOME_TYPE"]=="Pensioner","OCCUPATION_TYPE"] = "Pension"
df_application_record.loc[df_application_record["NAME_INCOME_TYPE"]=="Commercial associate","OCCUPATION_TYPE"] = "Commercial associate"
df_application_record.loc[df_application_record["NAME_INCOME_TYPE"]=="State servant","OCCUPATION_TYPE"] = "State servant"
df_application_record.loc[df_application_record["NAME_INCOME_TYPE"]=="Student","OCCUPATION_TYPE"] = "Student"
df_application_record = df_application_record.dropna()
df_application_record.isna().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
STATUS                 0
APPROVED               0
dtype: int64

In [24]:
df_application_record['Work_Time'] = -(df_application_record['DAYS_EMPLOYED'])//365

df_application_record = df_application_record.drop(df_application_record[df_application_record['Work_Time']>50].index)
df_application_record = df_application_record.drop(df_application_record[df_application_record['Work_Time']<0].index)
# df_application_record['Work_Time'].plot(kind='hist',bins=20,density=True)
df_application_record = df_application_record.drop(columns=['STATUS'])
df_application_record.drop(['DAYS_EMPLOYED'],axis=1,inplace=True)
df_application_record.head()


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,APPROVED,Work_Time
33,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,1,0,0,0,Security staff,2.0,-4,1,3
49,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,1,0,1,1,Commercial associate,1.0,-4,1,8
70,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,1,0,1,1,Commercial associate,1.0,-4,1,8
145,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,1,1,1,1,Accountants,2.0,-4,1,2
148,5112956,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,1,1,1,1,Accountants,2.0,-4,1,2


In [25]:
baseline_date = pd.to_datetime('2023-01-01')
df_application_record['BIRTH_DATE'] = baseline_date + pd.to_timedelta(df_application_record['DAYS_BIRTH'], unit='D')
df_application_record['AGE'] = (baseline_date - df_application_record['BIRTH_DATE']).dt.days // 365
df_application_record = df_application_record.drop(columns=['DAYS_BIRTH','BIRTH_DATE'])
df_application_record.isna().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
APPROVED               0
Work_Time              0
AGE                    0
dtype: int64

In [26]:
categorical_columns = ['CODE_GENDER', 'NAME_FAMILY_STATUS', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
dummy_columns = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
categorical_df = df_application_record[categorical_columns]
categorical_df = categorical_df.apply(lambda x: pd.factorize(x)[0])
categorical_df = pd.DataFrame(categorical_df)
df_application_record[categorical_columns] = categorical_df
df_application_record = pd.get_dummies(df_application_record, columns=dummy_columns)

print(df_application_record['APPROVED'].value_counts())
df_application_record.to_csv('dataset.csv', index=False)
print(df_application_record.columns)
df_application_record.head()


APPROVED
1    14693
0      300
Name: count, dtype: int64
Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_FAMILY_STATUS', 'FLAG_MOBIL',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS',
       'MONTHS_BALANCE', 'APPROVED', 'Work_Time', 'AGE',
       'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner',
       'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student',
       'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Academic degree',
       'NAME_EDUCATION_TYPE_Higher education',
       'NAME_EDUCATION_TYPE_Incomplete higher',
       'NAME_EDUCATION_TYPE_Lower secondary',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'NAME_HOUSING_TYPE_Co-op apartment',
       'NAME_HOUSING_TYPE_House / apartment',
       'NAME_HOUSING_TYPE_Municipal apartment',
       'NAME_HOUSING_TYPE_Office apartment',
       'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_Wi

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,...,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Pension,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_State servant,OCCUPATION_TYPE_Student,OCCUPATION_TYPE_Waiters/barmen staff
33,5008806,0,0,0,0,112500.0,0,1,0,0,...,False,False,False,False,False,False,True,False,False,False
49,5008810,1,1,0,0,270000.0,1,1,0,1,...,False,False,False,False,False,False,False,False,False,False
70,5008811,1,1,0,0,270000.0,1,1,0,1,...,False,False,False,False,False,False,False,False,False,False
145,5008815,0,0,0,0,270000.0,0,1,1,1,...,False,False,False,False,False,False,False,False,False,False
148,5112956,0,0,0,0,270000.0,0,1,1,1,...,False,False,False,False,False,False,False,False,False,False


# 创建数据集

In [27]:
# scaler = MinMaxScaler()
# df_application_record['AMT_INCOME_TOTAL']=scaler.fit_transform(df_application_record['AMT_INCOME_TOTAL'].values.reshape(-1, 1))
# df_application_record['DAYS_EMPLOYED']=scaler.fit_transform(df_application_record['DAYS_EMPLOYED'].values.reshape(-1, 1))
# df_application_record['MONTHS_BALANCE']=scaler.fit_transform(df_application_record['MONTHS_BALANCE'].values.reshape(-1, 1))
# scaler = StandardScaler()
# df_application_record['CNT_FAM_MEMBERS']=scaler.fit_transform(df_application_record['CNT_FAM_MEMBERS'].values.reshape(-1, 1))
# df_application_record['AGE']=scaler.fit_transform(df_application_record['AGE'].values.reshape(-1, 1))

negative_data_orgin = df_application_record[df_application_record['APPROVED']==0]
negative_data = negative_data_orgin.drop(['APPROVED', 'ID','CODE_GENDER'], axis = 1)

X = df_application_record.drop(['APPROVED', 'ID','CODE_GENDER'], axis = 1) 
y = df_application_record['APPROVED']
X = np.array(X,dtype=float)
y = np.array(y, dtype=int)
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)

negative_data = scalar.fit_transform(np.array(negative_data,dtype=float))

# # 创建 RandomUnderSampler 对象
# undersampler = RandomUnderSampler(sampling_strategy='majority')

# # 使用 RandomUnderSampler 来生成平衡的训练集
# X_train_under_random, y_train_under_random = undersampler.fit_resample(X_train, y_train)

# # 创建RandomOverSampler对象
# oversampler = RandomOverSampler(sampling_strategy='minority')

# # 使用RandomOverSampler来生成平衡的训练集
# X_train_over_random, y_train_over_random = oversampler.fit_resample(X_train, y_train)

# # 创建TomekLinks对象
# undersampler = TomekLinks()

# # 使用TomekLinks来生成平衡的训练集
# X_train_under_tomelinks, y_train_under_tomelinks = undersampler.fit_resample(X_train, y_train)

# 创建SMOTE对象
smote = SMOTE(sampling_strategy='minority')

# 使用SMOTE来生成平衡的训练集
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

(14993, 51)


In [28]:
data_list = [
             (X_train, y_train, "Original Data"),
            #  (X_train_over_random,y_train_over_random, "Over-sampled Data"),
            #  (X_train_under_random,y_train_under_random, "Under-sampled Data"),
            #  (X_train_under_tomelinks,y_train_under_tomelinks, "Tomelinks Data"),
            #  (X_train_smote,y_train_smote, "SMOTE Data")
            ]


# 特征提取

In [29]:
model_list = []
for X_train_processed, _, method_name in data_list:
   print("Training for the method: " + method_name)
   model = train_vae_anomaly_detection(X_train=X_train_processed,X_test=X_test, progress=True,num_epoch=250).eval()
   model_list.append(model)


Training for the method: Original Data


Epoch 1/250:   0%|          | 0/3 [00:00<?, ?it/s]

Best epoch: 249


In [30]:
total_list = [ t+ (data,)  for t, data in zip(data_list, model_list)]

In [31]:
# for X_train_processed, _, method_name, model in model_list:
#     model.eval()
#     with torch.no_grad():
#         encoded_data = model.encoder(torch.Tensor(X_train_processed).to(device))
#         encoded_data = encoded_data.cpu().numpy()
#         tsne = TSNE(n_components=2)
#         reduced_data = tsne.fit_transform(encoded_data)

# plt.scatter(reduced_data[:, 0], reduced_data[:, 1])
# plt.title("VAE Visualization")
# plt.show()

In [32]:


index = 0
neagtive_label_list = np.zeros(len(negative_data))
for X_train_processed, _, method_name, __ in total_list:
    predictions = model_list[index].predict_anomaly(torch.Tensor(X_test).to(device),threshold=0.05).detach().cpu().numpy()
    negative_data_pred = model_list[index].predict_anomaly(torch.Tensor(negative_data).to(device),threshold=0.05).detach().cpu().numpy()
    print(classification_report(y_test, predictions))
    print(classification_report(neagtive_label_list, negative_data_pred))
    index+=1

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        74
           1       0.98      1.00      0.99      3675

    accuracy                           0.98      3749
   macro avg       0.49      0.50      0.50      3749
weighted avg       0.96      0.98      0.97      3749

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     300.0
         1.0       0.00      0.00      0.00       0.0

    accuracy                           0.00     300.0
   macro avg       0.00      0.00      0.00     300.0
weighted avg       0.00      0.00      0.00     300.0



d:\Docu\DateMining\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Docu\DateMining\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Docu\DateMining\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Docu\DateMining\.venv\Lib\site-packages\sklearn\metr